In [ ]:
library(tidyverse)
library(data.table)
library(plotly) # for interactive ploting
library(DT) # for interactive tabulation
library(broom) # for tidy statistical summaries
library(caret) # for regression performance measures

In [ ]:
options(repr.matrix.max.rows=20, repr.matrix.max.cols=15) # for limiting the number of top and bottom rows of tables printed 

In [ ]:
datapath <- "~/data_ad454"

# Simple Linear Regression on IMF WEO Dataset

Let's first import the objects for the WEO dataset: 

In [ ]:
# wide data with features in the columns and countries/years in the rows
weo_wide2 <- readRDS(sprintf("%s/rds/01_01_weo_wide2.rds", datapath))

In [ ]:
weo_countries <- readRDS(sprintf("%s/rds/01_01_weo_countries.rds", datapath))
weo_subject <- readRDS(sprintf("%s/rds/01_01_weo_subject.rds", datapath))

Remember the nice widget to navigate through and search in tabular data:

In [ ]:
weo_subject %>% datatable(
  filter = "top",
  options = list(pageLength = 20)
)

You are supposed to create a simple linear regression model to estimate the "NGDP_RPCH" feature for 2019 data in weo_wide2.

What does that NGDP_TPCH stand for:

In [ ]:
weo_subject[WEO_Subject_Code == "NGDP_RPCH"]

You should select a reasonable independent variable from the available features to explain this dependent variable.

The independent variable should not be any of the following ones:

In [ ]:
weo_subject[str_detect(Subject_Descriptor, "Gross domestic product")]

So the independent variable can be any one of the following:

In [ ]:
weo_subject[!str_detect(Subject_Descriptor, "Gross domestic product")]  %>% datatable(
  filter = "top",
  options = list(pageLength = 20)
)

Or you may create a new feature using multiple allowed features above through any kind of mathematical transformations. Note that only one independent variable can be included in the right hand side of the model

You may want to inspect the pairwise relationship among multiple features visually to detect the candidates to be included in the model. Some possible tools are as follows:

- pairs()
- psych::pairs.panels()
- GGally::ggpairs()

Total missing cases (NA's) of the variable(s) (directly as an independent variable or indirectly for calculating a new feature) that you use should not be more than 30. You can check that with `function(x) sum(is.na(x))` for a column

The steps you are required to follow are as follows:

- Show the calculations to create a new feature as an independent variable if you do so (if you include a feature directly skip this step)
- Filter for year 2019 and select the dependent and independent variables
- Create a scatterplot between the pair including a best fit line
- Split the dataset randomly into train and test partitions. The ratio of train partition to the overall set is supposed to be between 0.5 and 0.7
- Create and run a simple linear regression model and assign the model into a named object
- Print the summary of the model and interpret with a few words. **Note that the coefficient of the independent variable should be significantly different than 0 at 0.05 significance level. If not please select another independent variable**
- Create four vectors for predictions/actual values of train/test datasets
- Print two scatter plots: 1) predictions vs actual values of train set, 2) predictions vs actual values of test set. You may need to combine the predictions and actual values into a data.table with appropriate names (one data.table for train, one data.table for the test set) in order to feed into ggplot + geom_point. Please also add a diagonal line and main title. 
- Compare the R2 and RMSE of train and test datasets and interpret the comparison with a few words

**Hint: Most steps follow the 03_02_simple_regression.ipynb notebook**

# Answer

In [ ]:
weo_subset <- weo_wide2 %>% filter(year == 2019) %>%
select(NGDP_RPCH, NID_NGDP) %>% na.omit

In [ ]:
weo_subset[, lapply(.SD, function(x) sum(is.na(x)))]

In [ ]:
weo_subset %>% ggplot(aes(x = NID_NGDP,
                           y = NGDP_RPCH)) +
                        geom_point() +
geom_smooth(method = "lm", formula = y ~ x, se = F)

In [ ]:
train_ratio <- 0.7

Randomly create row indices for train partition

In [ ]:
train_indices <- weo_subset[,sample(.N * train_ratio)]

Split the data into two partitions

In [ ]:
train_data <- weo_subset[train_indices]
test_data <- weo_subset[-train_indices]

Check whether partitions are mutually exclusive:

In [ ]:
weo_subset[,.N]
train_data[,.N]
test_data[,.N]

In [ ]:
model1 <- lm(NGDP_RPCH ~ NID_NGDP, data = train_data)

See the summary:

In [ ]:
summary(model1)

tidy() from broom package extract and present useful information from the model in a tabulated manner:

In [ ]:
tidy(model1)

In [ ]:
actual_train <- train_data$NGDP_RPCH
predicted_train <- predict(model1, train_data)

In [ ]:
actual_test <- test_data$NGDP_RPCH
predicted_test <- predict(model1, test_data)

In [ ]:
model_dt <- data.table(partition = c("train", "test"),
                       R2 = c(R2(predicted_train, actual_train),
                                R2(predicted_test, actual_test)),
                        RMSE = c(RMSE(predicted_train, actual_train),
                                 RMSE(predicted_test, actual_test)),
                        MAE = c(MAE(predicted_train, actual_train),
                                MAE(predicted_test, actual_test))
                        )

In [ ]:
model_dt

In [ ]:
data.table(actual = actual_train, predictions = predicted_train) %>%
ggplot(aes(x = actual, y = predictions)) +
geom_point() +
geom_abline(slope = 1, intercept = 0) +
ggtitle("Train Actual vs. Predictions")

In [ ]:
data.table(actual = actual_test, predictions = predicted_test) %>%
ggplot(aes(x = actual, y = predictions)) +
geom_point() +
geom_abline(slope = 1, intercept = 0) +
ggtitle("Test Actual vs. Predictions")